In [1]:
sc.addPyFile("util.py")
sc.addPyFile("linear_regression.py")

import numpy as np
import linear_regression
import logging
import multiprocessing
import threading
from util import get_train_X_Y

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
ps_hosts = [ "localhost:30000" ]
worker_hosts = [ "localhost:30001"]

In [3]:
def train(ps_hosts, worker_hosts, idx, partition, job_name):
    logging.debug("****** Start to train the model")

    pool = multiprocessing.Pool(processes=1)
    if job_name == "ps":
        pool.apply_async(linear_regression.distributed_train, (ps_hosts, worker_hosts, "ps", idx, None, None))
        #linear_regression.distributed_train(ps_hosts, worker_hosts, "ps", idx, None, None)
        pool.close()
        #pool.join()
    else:
        X, Y = get_train_X_Y(partition)
        pool.apply_async(linear_regression.distributed_train, (ps_hosts, worker_hosts, "worker", idx, X, Y))
        #linear_regression.distributed_train(ps_hosts, worker_hosts, "worker", idx, X, Y)
        pool.close()
        #pool.join()
    return ["Task # %d done".format(idx)]

In [4]:
worker_rdd = sc.parallelize(np.random.random(1000)*100, len(worker_hosts)).map(lambda x: (x, x * 0.68 + 1.2))
ps_rdd = sc.parallelize(ps_hosts, len(ps_hosts))

In [5]:
ps_rdd.mapPartitionsWithIndex(lambda idx, it: train(ps_hosts, worker_hosts, idx, it, "ps")).count()

1

In [6]:
worker_rdd.mapPartitionsWithIndex(lambda idx, it: train(ps_hosts, worker_hosts, idx, it, "worker")).count()

1